In [ ]:
import uuid
import asyncio
import pandas as pd
from typing import List, Dict
from uuid import UUID
from tqdm import tqdm
from rich import print
from rich.console import Console
from rich.table import Table
from order_book_simulator.config import OrderRequest, Priority, Side
from order_book_simulator.config.trade import Trade
from order_book_simulator.core import OrderBook, MatchingEngine
from order_book_simulator.websocket import ConnectionManager
from order_book_simulator.analysis import MetricsCalculator
from order_book_simulator.util import create_trade_table

console = Console()

In [2]:
orders_df = pd.read_csv('../data/orders-1.csv')

In [3]:
# just to adhere to my definition of the OrdrerRequest dataclass
# unix timestamp is more efficient for computation and storage
# trader_uuid is more secure and flexible than group number
# (in real life, traders won't identify themselves with group numbers)
# so we convert them here
# Note: keeping full nanosecond precision by converting to float (seconds with decimals)

orders_df["unix_timestamp"] = pd.to_datetime(orders_df["Timestamp"]).astype(int) / 10**9 # preserve nanoseconds
group_to_uuid = {group: uuid.uuid4() for group in orders_df["Group Number"].unique()}
orders_df["trader_uuid"] = orders_df["Group Number"].map(group_to_uuid)

# Create reverse mapping for displaying trades
uuid_to_group = {v: k for k, v in group_to_uuid.items()}

In [4]:
order_book = OrderBook()
metrics_calculator = MetricsCalculator(order_book, 0.01)
connection_manager = ConnectionManager()

# websocket related stuff are not required for this assignment but we designed some functions to allow that
matching_engine = MatchingEngine(order_book, 
                                 connection_manager, # not needed for assignment
                                 asyncio.get_event_loop(), # not needed for assignment
                                 metrics_calculator)

In [ ]:
# Process orders and collect trades
filled_trades: List[Trade] = []
total_orders = len(orders_df)
checkpoints = {4, 19, total_orders - 1}

for index, row in tqdm(orders_df.iterrows(), total=total_orders, desc="Processing orders"):
    order_request = OrderRequest(
        trader_id=row["trader_uuid"],
        side=row["Buy Sell Direction"],
        priority=Priority.HIGH,
        timestamp=row["unix_timestamp"],
        price=row["Order Price"],
        quantity=row["Quantity"]
    )
    
    trades = matching_engine.process_order(order_request)
    if trades:
        filled_trades.extend(trades)
    
    # Display order book at checkpoints
    if index in checkpoints:
        label = "FINAL" if index == total_orders - 1 else f"#{index + 1}"
        console.print(f"\n[bold cyan]=== Order Book after {label} orders ===[/bold cyan]")
        print(order_book)

# Display all executed trades
console.print("\n[bold cyan]=== ALL EXECUTED TRADES ===[/bold cyan]")
console.print(f"[bold]Total number of trades: {len(filled_trades)}[/bold]\n")
console.print(create_trade_table(filled_trades, uuid_to_group))

Processing orders:   0%|          | 0/248 [00:00<?, ?it/s]

=== Order Book after #5 orders ===

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Bid Price ┃ Bid Quantity ┃ Ask Price ┃ Ask Quantity ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│     11.00 │           20 │     12.00 │           30 │
│           │              │     50.00 │           11 │
└───────────┴──────────────┴───────────┴──────────────┘

=== Order Book after #20 orders ===

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Bid Price ┃ Bid Quantity ┃ Ask Price ┃ Ask Quantity ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│     11.00 │           50 │     12.00 │           55 │
│     10.00 │           75 │     50.00 │           11 │
└───────────┴──────────────┴───────────┴──────────────┘

...

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Bid Price ┃ Bid Quantity ┃ Ask Price ┃ Ask Quantity ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│     10.00 │          190 │     11.00 │           81 │
│      9.00 │          228 │     12.00 │          510 │
│      8.00 │           25 │     13.00 │          155 │
│      7.00 │          130 │     15.00 │           50 │
│           │              │     50.00 │           11 │
└───────────┴──────────────┴───────────┴──────────────┘

Processing orders: 100%|██████████| 248/248 [00:00<00:00, 6606.63it/s]



=== ALL EXECUTED TRADES ===

Total number of trades: 204

┏━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┓
┃ Trade # ┃ Quantity ┃  Price ┃ Maker Group ┃ Maker UUID             ┃ Taker Group ┃ Taker UUID            ┃ Side ┃
┡━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━┩
│       1 │       30 │ $11.00 │ 15          │ d8db39d8-d3d6-4e2c-87… │ 16          │ b4bb277a-1a92-4698-8… │ sell │
│       2 │       30 │ $11.00 │ 15          │ d8db39d8-d3d6-4e2c-87… │ 16          │ b4bb277a-1a92-4698-8… │ sell │
│       3 │       10 │ $11.00 │ 15          │ d8db39d8-d3d6-4e2c-87… │ 6           │ 4a42da52-9be1-40d8-8… │ sell │
│       4 │       10 │ $11.00 │ 15          │ d8db39d8-d3d6-4e2c-87… │ 6           │ 4a42da52-9be1-40d8-8… │ sell │
│       5 │       50 │ $10.00 │ 5           │ 361ef7c2-e97d-4b86-bf… │ 10          │ 5ecae8a0-42dd-45f1-a… │ sell │
│       6 │       30 │ $10.00 │ 5           │ 361ef7c2-e97d-4b86-bf… │ 10          │ 5ecae8a0-42dd-45f1-a… │ sell │
│       7 │       20 │ $11.00 │ 10          │ 5ecae8a0-42dd-45f1-ad… │ 3           │ 182320fb-3475-49de-b… │ buy  │
│       8 │        5 │ $12.00 │ 16          │ b4bb277a-1a92-4698-89… │ 3           │ 182320fb-3475-49de-b… │ buy  │
│       9 │       20 │ $11.00 │ 3           │ 182320fb-3475-49de-b0… │ 2           │ 99151189-cb6f-457c-8… │ sell │
│      10 │       20 │ $11.00 │ 3           │ 182320fb-3475-49de-b0… │ 12          │ a4107c07-6a0d-4710-8… │ sell │
│      11 │       10 │ $11.00 │ 3           │ 182320fb-3475-49de-b0… │ 2           │ 99151189-cb6f-457c-8… │ sell │
│      12 │       10 │ $11.00 │ 17          │ b698bca8-0c4e-4c78-b9… │ 2           │ 99151189-cb6f-457c-8… │ sell │
│      13 │       20 │ $11.00 │ 17          │ b698bca8-0c4e-4c78-b9… │ 12          │ a4107c07-6a0d-4710-8… │ sell │
│      14 │        5 │ $12.00 │ 16          │ b4bb277a-1a92-4698-89… │ 17          │ b698bca8-0c4e-4c78-b… │ buy  │
│      15 │       10 │ $11.00 │ 17          │ b698bca8-0c4e-4c78-b9… │ 2           │ 99151189-cb6f-457c-8… │ sell │
│      16 │       20 │ $11.00 │ 13          │ fa8a9258-6206-48fc-81… │ 2           │ 99151189-cb6f-457c-8… │ sell │
│      17 │       10 │ $10.00 │ 17          │ b698bca8-0c4e-4c78-b9… │ 2           │ 99151189-cb6f-457c-8… │ sell │
│      18 │       39 │ $10.00 │ 17          │ b698bca8-0c4e-4c78-b9… │ 2           │ 99151189-cb6f-457c-8… │ sell │
│      19 │       18 │ $11.00 │ 12          │ a4107c07-6a0d-4710-89… │ 5           │ 361ef7c2-e97d-4b86-b… │ buy  │
│      20 │        1 │ $10.00 │ 17          │ b698bca8-0c4e-4c78-b9… │ 18          │ 0dc93dfc-ebf7-47de-b… │ sell │
│      21 │       25 │ $10.00 │ 3           │ 182320fb-3475-49de-b0… │ 18          │ 0dc93dfc-ebf7-47de-b… │ sell │
│      22 │        4 │ $10.00 │ 11          │ 061bc19c-3e99-4c5a-a7… │ 18          │ 0dc93dfc-ebf7-47de-b… │ sell │
│      23 │        8 │ $11.00 │ 12          │ a4107c07-6a0d-4710-89… │ 1           │ f8d93e64-87de-4a47-b… │ buy  │
│      24 │       20 │ $10.00 │ 11          │ 061bc19c-3e99-4c5a-a7… │ 4           │ 25a6b711-b285-483f-9… │ sell │
│      25 │        3 │ $11.00 │ 12          │ a4107c07-6a0d-4710-89… │ 1           │ f8d93e64-87de-4a47-b… │ buy  │
│      26 │       30 │ $10.00 │ 11          │ 061bc19c-3e99-4c5a-a7… │ 20          │ 6643233d-7499-4b7a-b… │ sell │
│      27 │       46 │ $10.00 │ 11          │ 061bc19c-3e99-4c5a-a7… │ 5           │ 361ef7c2-e97d-4b86-b… │ sell │
│      28 │       40 │ $10.00 │ 7           │ d1a48546-375f-49f9-b8… │ 5           │ 361ef7c2-e97d-4b86-b… │ sell │
│      29 │       20 │ $10.00 │ 13          │ fa8a9258-6206-48fc-81… │ 5           │ 361ef7c2-e97d-4b86-b… │ sell │
│      30 │       50 │ $10.00 │ 9           │ 58feba90-005f-43d5-93… │ 5           │ 361ef7c2-e97d-4b86-b… │ sell │
│      31 │       10 │ $10.00 │ 5           │ 361ef7c2-e97d-4b86-bf… │ 16          │ b4bb277a-1a92-4698-8… │ buy  │
│      32 │       14 │ $10.00 │ 5           │ 361ef7c2-e

In [6]:
final_metrics = metrics_calculator.calculate_averages(orders_df["unix_timestamp"].max())

--- Final Metrics ---

⚠️  Total Crossed Book Occurrences: 12

Crossed Book Events:

Event 1:

Timestamp: 1759868528.65815

...

Time-Weighted Average:

Time-Weighted Avg Bid-Ask Spread (in Ticks): 93.2276

Time-Weighted Avg Bid-Ask Spread (in Basis Points): 897.7823

Time-Weighted Avg Best-Level Market Depth (in Dollars): $2,421.55

Time-Weighted Avg Order Imbalance: +0.3443